# Brain_model

In [1]:
import tensorflow as tf 
tf.version

<module 'tensorflow._api.v2.version' from 'C:\\Users\\Afrid\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

In [2]:
from tensorflow import keras 
from keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
model = keras.models.load_model('./brain_model/model2')

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='categorical',
    seed=RANDOM_SEED
)

# Lung_model

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import pickle
import numpy as np

In [15]:
def save_feature_vectors(model, loader, output_size=(1, 1), file="testb7"):
    model.eval()
    images, labels = [], []

    for idx, (x, y) in enumerate(tqdm(loader)):
        x = x.to(DEVICE)

        with torch.no_grad():
            features = model.extract_features(x)
            features = F.adaptive_avg_pool2d(features, output_size=output_size)
        images.append(features.reshape(x.shape[0], -1).detach().cpu().numpy())
        labels.append(y.numpy())

    np.save(f"./X_{file}.npy", np.concatenate(images, axis=0))
    np.save(f"./y_{file}.npy", np.concatenate(labels, axis=0))
    model.train()

In [16]:
# load efficientNet model
model = EfficientNet.from_pretrained("efficientnet-b7")
model._fc = nn.Linear(2560, 1)
model.load_state_dict(torch.load('./lung_model/lung_model.pt', map_location=torch.device('cpu')))
model.eval()

# load the clf model 
clf = pickle.load(open('./lung_model/lung_model.sav', 'rb'))

# input transforms
test_dir = './lung_model/val_data/'
DEVICE = 'cpu'
X_transform = T.Compose([
    T.Resize((256,256)),
    T.ToTensor()
])
input_ = ImageFolder(test_dir, transform=X_transform)
test_loader = DataLoader(input_)

# save feature vectors as npy file   
save_feature_vectors(model, test_loader, output_size=(1, 1), file="test_b7")

# load featrue vector npy file 
X_val = np.load('./X_test_b7.npy')
y_val = np.load('./y_test_b7.npy')

# classifier 
clf.score(X_val, y_val)
clf.predict(X_val)

Loaded pretrained weights for efficientnet-b7


C:\Users\Afrid\Anaconda3\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.51it/s]


array([0, 0, 0, 1, 1], dtype=int64)

# Cough_model

In [1]:
import numpy as np
import pandas as pd
import glob
import librosa
import pickle
from scipy import signal
import warnings
warnings.filterwarnings("ignore")

In [216]:
def feature_extraction(file_name):
    try:
        X, sr = librosa.load(file_name)
        if X.ndim > 1:
            X = X[:, 0]
        X = X.T
        stft = np.abs(librosa.stft(X))
        chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        rmse = np.mean(librosa.feature.rms(y=X))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=X, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=X, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(X))
        mfcc = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T, axis=0)  # 40 values
#         mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
        return np.mean(chroma_stft), rmse, spec_cent, spec_bw, rolloff, zcr, mfcc 
    except ValueError:
        return None

In [341]:
''' 
>    covid_status  (scale:1-7)	
                        'recovered_full', 'positive_mild', 'healthy', 
                        'resp_illness_not_identified', 'positive_asymp', 
                        'positive_moderate', 'no_resp_illness_exposed'
>    gender        (0 or 1)	
>    age           (15-85)
'''
audio1 = './cough_model/data/neg-0421-085-cough-m-43.wav'
audio2 = './cough_model/data/pos-0421-087-cough-f-40.wav'
audio3 = './cough_model/data/pos-0421-086-cough-m-65.wav'

data = pd.DataFrame([3, 0, 65]).T

features = pd.DataFrame(feature_extraction(audio3)).T
featured_csv = pd.concat([data, features], axis=1)

mfcc = pd.DataFrame()
for i in range(len(featured_csv)):
  _, __ , Sxx = signal.spectrogram(featured_csv[6][i])
  a = pd.DataFrame(Sxx.T)
  mfcc = pd.concat([mfcc, a])

df = pd.concat([featured_csv.drop([6], axis=1), mfcc], axis=1)

filename = './cough_model/xgb.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(df)
result

array([1], dtype=int64)

,0,1,2
0,3,0,65


In [342]:
# neg=0, pos=1
loaded_model.predict_proba(df)

array([[0.00305234, 0.99694766]])

# Streamlit